In [7]:
import os
import shutil
import pandas as pd

# Dataset paths
data_dir = 'Data/input/images_resized'

# Create necessary directories
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Split data into training and validation sets
image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg') or f.endswith('.png')]
num_train = int(0.8 * len(image_files))
train_files = image_files[:num_train]
val_files = image_files[num_train:]

for file_name in train_files:
    shutil.move(os.path.join(data_dir, file_name), train_dir)
for file_name in val_files:
    shutil.move(os.path.join(data_dir, file_name), val_dir)

# Create dummy annotations (assuming bounding boxes around entire images)
annotations = []

def create_annotations(image_dir, label_dir):
    for image_file in os.listdir(image_dir):
        if image_file.endswith('.jpg') or image_file.endswith('.png'):
            annotations.append({
                'image': image_file,
                'bbox': [0, 0, 1, 1],  # Dummy bounding box (whole image)
                'category_id': 1  # Category 1 means 'insect'
            })

create_annotations(train_dir, 'train')
create_annotations(val_dir, 'val')

# Save annotations to CSV
df = pd.DataFrame(annotations)
df.to_csv('/content/yolov5/data/annotations.csv', index=False)


OSError: Cannot save file into a non-existent directory: '\content\yolov5\data'

In [ ]:
# Create a custom dataset YAML file
dataset_yaml = """
path: /content/yolov5/data
train: images/train
val: images/val

# number of classes
nc: 1

# class names
names: ['insect']
"""

with open('/content/yolov5/data/custom_dataset.yaml', 'w') as f:
    f.write(dataset_yaml)

# Train the YOLOv5 model
!python train.py --img 640 --batch 16 --epochs 50 --data /content/yolov5/data/custom_dataset.yaml --cfg yolov5s.yaml --weights yolov5s.pt --name custom_yolov5

# Evaluate the model
!python detect.py --weights runs/train/custom_yolov5/weights/best.pt --img 640 --conf 0.25 --source /mnt/data/20200809000003-1.jpg


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import torch

# Load the trained model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/custom_yolov5/weights/best.pt')

# Perform detection on a new image
image_path = '/mnt/data/20200809000003-1.jpg'
results = model(image_path)

# Display the results
results.show()
